# Kernel parameters nan error（demo）
Date: 2024-3-11<br/>
Author: zhenjie<br/>
## 1. 问题定义
在使用GAR、CIGAR的过程中，涉及到高维大量数据的情况下，有时候数据中有的数值偏离分布，或者是在梯度计算的时候存在问题，导致Kernel的参数出现nan的情况。<br/>
下面我们将通过一个简单的例子来说明这个问题。

In [ ]:
import torch
import torch.nn as nn

EPS = 1e-9
class SquaredExponentialKernel(nn.Module):
    """
    Squared Exponential kernel module with scalar length scale.

    Args:
        length_scale (float): The length scale value. Default is 1.0.
        signal_variance (float): The signal variance value. Default is 1.0.

    Attributes:
        length_scale (nn.Parameter): The length scale.
        signal_variance (nn.Parameter): The signal variance.

    """

    def __init__(self, length_scale=1.0, signal_variance=1.0):
        super().__init__()
        self.length_scale = nn.Parameter(torch.tensor([length_scale])) #log
        self.signal_variance = nn.Parameter(torch.tensor([signal_variance]))

    def forward(self, x1, x2):
        """
        Compute the covariance matrix using the squared exponential kernel.

        Args:
            x1 (torch.Tensor): The first input tensor.
            x2 (torch.Tensor): The second input tensor.

        Returns:
            torch.Tensor: The covariance matrix.

        """
        
        x1 = x1.reshape(x1.shape[0], -1)
        x2 = x2.reshape(x2.shape[0], -1)

        sqdist = torch.sum(x1**2, 1).reshape(-1, 1) + torch.sum(x2**2, 1) - 2 * torch.matmul(x1, x2.T)
        return self.signal_variance.exp().pow(2) * torch.exp(-0.5 * sqdist / self.length_scale.exp().pow(2))

## 2. 解决方法
导致这个问题的原因很多，有可能是数据的问题，也有可能是梯度计算的问题。我们可以通过以下几种方法来分析这个问题：<br/>
1. 数据分析：通过数据分析，我们可以找出数据中的异常值，然后通过一些方法来处理异常值，比如删除异常值，或者是通过一些插值的方法来处理异常值。<br/>
2. 梯度检查：通过梯度检查，我们可以利用pytorch中的tensorboard找出梯度计算的问题，进一步分析原因。<br/>
3. 模型调参：通过调整模型的参数，可能可以避免这个问题。<br/>

上述方法可能可以解决这个问题，但是在分析的过程中比较复杂，经过试验，我们发现在GP的训练过程中，参数的波动不是很大，所以我们可以重置参数的方法来解决这个问题。下面是我们的解决方法：<br/>


In [ ]:
class SquaredExponentialKernel(nn.Module):
    """
    Squared Exponential kernel module with scalar length scale.

    Args:
        length_scale (float): The length scale value. Default is 1.0.
        signal_variance (float): The signal variance value. Default is 1.0.

    Attributes:
        length_scale (nn.Parameter): The length scale.
        signal_variance (nn.Parameter): The signal variance.

    """

    def __init__(self, length_scale=1.0, signal_variance=1.0):
        super().__init__()
        self.length_scale = nn.Parameter(torch.tensor([length_scale])) #log
        self.signal_variance = nn.Parameter(torch.tensor([signal_variance]))

    def forward(self, x1, x2):
        """
        Compute the covariance matrix using the squared exponential kernel.

        Args:
            x1 (torch.Tensor): The first input tensor.
            x2 (torch.Tensor): The second input tensor.

        Returns:
            torch.Tensor: The covariance matrix.

        """
        ## When encountering abnormal data or gradients causing parameter errors, parameter reset can achieve good training results
        if torch.isnan(self.length_scale):
            self.length_scale = nn.Parameter(torch.tensor([1.0])).to(x1.device)
        if torch.isnan(self.signal_variance):
            self.signal_variance = nn.Parameter(torch.tensor([1.0])).to(x1.device)
        
        x1 = x1.reshape(x1.shape[0], -1)
        x2 = x2.reshape(x2.shape[0], -1)

        sqdist = torch.sum(x1**2, 1).reshape(-1, 1) + torch.sum(x2**2, 1) - 2 * torch.matmul(x1, x2.T)
        return self.signal_variance.exp().pow(2) * torch.exp(-0.5 * sqdist / self.length_scale.exp().pow(2))